In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/AdaFace/
# !ls

In [3]:
# %pip install -r requirements.txt

In [4]:
import sys
import torch
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")

Python version: 3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)], sys.version_info(major=3, minor=11, micro=0, releaselevel='final', serial=0) 
Pytorch version: 2.0.0+cu118 


In [5]:
# !python build_database.py --database_savedir './AdaFace/face_database'

In [6]:
sys.path.append('./AdaFace/')
import net
import cv2
import torch
import os
from yolo_face.align import YOLO_FACE
import numpy as np
import matplotlib.pyplot as plt
import time
import argparse

In [7]:
adaface_models = {
    'ir_50':"pretrained/adaface_ir50_ms1mv2.ckpt",
    'ir_101':"pretrained/adaface_ir101_ms1mv2.ckpt",
}
def load_pretrained_model(architecture='ir_50',path=''):
    # load model and pretrained statedict
    assert architecture in adaface_models.keys()
    model = net.build_model(architecture)
    pt = path if path is  not None else adaface_models[architecture]
    statedict = torch.load(pt)['state_dict']
    model_statedict = {key[6:]:val for key, val in statedict.items() if key.startswith('model.')}
    model.load_state_dict(model_statedict)
    model.eval()
    return model

def to_input(pil_rgb_image,RGB=True):
    np_img = np.array(pil_rgb_image)
    if RGB:
        brg_img = ((np_img / 255.) - 0.5) / 0.5
    else :
        brg_img = ((np_img[:,:,::-1] / 255.) - 0.5) / 0.5
    tensor = torch.tensor(np.expand_dims(brg_img.transpose(2,0,1),axis=0)).float()
    return tensor

In [8]:
%pwd

'c:\\Users\\User\\Desktop\\study\\_ML\\AdaFace'

In [9]:
fr_weights = 'c:/Users/User/Desktop/study/_ML/AdaFace/pretrained/adaface_ir50_ms1mv2.ckpt'
fd_weights = 'c:/Users/User/Desktop/study/_ML/AdaFace/pretrained/yolov7-tiny.pt'
isyolo = True
arch = 'ir_50'
database_path = 'face_database'
database_dir =  './AdaFace/face_database' #validation sets
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_pretrained_model(arch,path=fr_weights).to(device)
print('face recongnition model loaded')
yoloface = YOLO_FACE(fd_weights,device=device) if isyolo else None
print('face detection model loaded')
torch.set_grad_enabled(False)

c:\Python311\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


face recongnition model loaded
face detection model loaded


# Firebase Firestore Database Initialization & Function Definition
- create_class(): no need to call by user
- student_arrive(): detect a student's face at the moment, parameter is student's ID.

In [10]:
# %pip install --user firebase-admin

In [11]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from datetime import datetime
import pytz
import asyncio

cred = credentials.Certificate("./serviceAccountKey.json")
firebase_admin.initialize_app(cred)


# 初始化firestore
db = firestore.client()

In [12]:

def create_class(class_date):
    student_id_list_ref = db.collection("students").document("student_id_list")
    class_doc_ref = db.collection("classes").document(str(class_date))
    class_doc_ref.set(student_id_list_ref.get().to_dict())

def student_arrive(student_id):
    cur_date = datetime.now(pytz.timezone('Asia/Taipei')).strftime("%m%d")
    cur_time = datetime.now(pytz.timezone('Asia/Taipei')).strftime("%H:%M:%S")
    class_ref = db.collection("classes").document(str(cur_date))
    snapshot = class_ref.get()
    snapshot_dict = snapshot.to_dict()
    class_update_data = {
        student_id: cur_time
    }
    if class_ref.get().exists == False:
        create_class(cur_date)
    elif type(snapshot_dict[student_id]) == str:
        print(f"{student_id} had arrived at {snapshot_dict[student_id]}")
        return
    class_ref.update(class_update_data)
    ##########
    student_document_ref = db.collection("students").document("student_id_list").collection("ML").document(student_id)
    student_update_data = {
        cur_date : cur_time
    }
    if student_document_ref.get().exists == False:
        student_document_ref.set(student_update_data)
    else:
        student_document_ref.update(student_update_data)

    print(f"update successed. {student_id} arrives at {cur_date} {cur_time}")

# INPUT

## WebCam setting
[REFERENCE](https://colab.research.google.com/github/OmniXRI/Colab_Webcam_OpenCV/blob/main/Colab_Webcam_OpenCV.ipynb#scrollTo=nLxYK8ccMMI-)


In [13]:
# https://colab.research.google.com/github/OmniXRI/Colab_Webcam_OpenCV/blob/main/Colab_Webcam_OpenCV.ipynb#scrollTo=nLxYK8ccMMI-
from IPython.display import display, Javascript, Image
# from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import subprocess
import os

In [14]:
def js_to_image(js_reply):
  # 解碼成 base64 格式影像
  image_bytes = b64decode(js_reply.split(',')[1])
  # 轉換影像變成 Numpy 格式
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # 解碼 Numpy格式到 OpenCV BGR 影像格式
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

def image_to_js(cv2_img):
  _, buffer = cv2.imencode('.jpg', cv2_img)
  image_str = b64encode(buffer).decode('utf-8')

  # show img
  display(Javascript("""
      var img = document.createElement('img');
      img.src = 'data:image/jpeg;base64,{}';
      document.body.appendChild(img);
  """.format(image_str)))

def img_process(img):

    bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    return bgr

In [15]:
def video_frame(label, bbox):
  # data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  result = subprocess.run(['node', 'c:/Users/User/Desktop/study/_ML/AdaFace/video_stream.js', label, bbox], capture_output=True, text=True)
  print("stdout:", result.stdout)
  print("stderr:", result.stderr)
  result.check_returncode()
  data = result.stdout.strip()
  return data

## Create Video Stream and Start Detection

now detect every 5 seconds
want to improve to approximately real-time
and improve the representation in the scheme

-

想要可以把人框起來上面寫人名跟score的那種酷效果
可能要改yolo_face回傳的參數

yolo_face裡面有個detect video還是什麼的但那在colab上用不了Q

-

1201 update:

1. 可以一次測兩個人(歡呼)但我沒有第二台手機沒辦法測第三個人，兩個人ok的話三個人應該也行吧

2. firestore database只會紀錄第一次arrive的時間


In [16]:
import time
from pathlib import Path

import os
import copy
import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from yolo_face.models.experimental import attempt_load
from yolo_face.utils.datasets import LoadStreams, LoadImages
from yolo_face.utils.general import check_img_size, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path, save_one_box
from yolo_face.utils.plots import colors, plot_one_box
from yolo_face.utils.torch_utils import select_device, load_classifier, time_synchronized

class Opt:
    def __init__(self, weights='yolo_face/yolov7-tiny.pt', source='0', img_size=640, conf_thres=0.25,
                iou_thres=0.45, device='', view_img=False, save_txt=False, save_txt_tidl=False,
                save_bin=False, save_conf=False, save_crop=False, nosave=False, classes=None,
                agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp',
                exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, kpt_label=5):
        self.weights = weights
        self.source = source
        self.img_size = img_size
        self.conf_thres = conf_thres
        self.iou_thres = iou_thres
        self.device = device
        self.view_img = view_img
        self.save_txt = save_txt
        self.save_txt_tidl = save_txt_tidl
        self.save_bin = save_bin
        self.save_conf = save_conf
        self.save_crop = save_crop
        self.nosave = nosave
        self.classes = classes
        self.agnostic_nms = agnostic_nms
        self.augment = augment
        self.update = update
        self.project = project
        self.name = name
        self.exist_ok = exist_ok
        self.line_thickness = line_thickness
        self.hide_labels = hide_labels
        self.hide_conf = hide_conf
        self.kpt_label = kpt_label

opt = Opt()
# print(opt)


# with torch.no_grad():
#     if opt.update:  # update all models (to fix SourceChangeWarning)
#         for opt.weights in ['yolov5s.pt', 'yolov5m.pt', 'yolov5l.pt', 'yolov5x.pt']:
#             detect(opt=opt)
#             strip_optimizer(opt.weights)
#     else:
#         detect(opt=opt)


In [17]:
# 啟動網路攝影機開始接收串流影像
# video_stream()
from detect import detect
cap = cv2.VideoCapture(cv2.CAP_DSHOW)
# 設定串流影像文字標籤
label_html = 'Capturing...'
# 清空疊合影像內容
overlap_img = ''
source = './test_images'
features = []
face=[]
imgs=[]
test_image = sorted(os.listdir(source))
ct=0
# 執行取像、處理循環，執行過程中若於影像上按下滑鼠鍵即會中止程式
cam_start_time = time.time()
while True:
    # 從網路攝影機取得串流影像目前影格
    ret, frame = cap.read()
    # 若無法取得影格則結束循環
    if not ret:
        print("nothing")
        break
    
    cur_time = time.time()

    # if cur_time - cam_start_time >= 5:
    #     cam_start_time = cur_time

    if isyolo:
        aligned_bgr_imgs, coordinate = yoloface.frame_detect(frame)
        print("Detected faces number: " , len(aligned_bgr_imgs))
        # print("type: ", type(aligned_bgr_imgs))
        for i,img in enumerate(aligned_bgr_imgs) :
            # imgs.append(tt[i])
            # cv2.imwrite(f'deteced_{ct}_{i}.jpg',img)
            bgr_tensor_input = to_input(img,True).to(device)
            feature, _ = model(bgr_tensor_input)
            features.append(feature)
            face.append(f'deteced_{ct}_{i}.jpg')
        ct+=1
    else:
        # aligned_rgb_img = align.get_aligned_face(js_img, isarray=True)
        aligned_rgb_img = align.get_aligned_face(frame, isarray=True)
        if(aligned_rgb_img is not None):
            bgr_tensor_input = to_input(aligned_rgb_img,False).to(device)
            feature, _ = model(bgr_tensor_input)
            features.append(feature)

    # if nothing detected
    if features == []:
        continue

    database_dir =  'c:/Users/User/Desktop/study/_ML/AdaFace/face_database'
    database_pt_dir = os.path.join(database_dir,'database.pt')
    if(os.path.exists(database_pt_dir)):
        database = torch.load(database_pt_dir).to(device)
        with open(os.path.join(database_dir, 'ID_list.txt'), 'r') as file:
            ID_list = [line.strip() for line in file.readlines()]
        start_time = time.time()
        with torch.no_grad():
            similarity_scores = torch.cat(features) @ database.T
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"computing time: {execution_time} seconds")
        score = similarity_scores.cpu().detach().numpy()
        max_index = np.argmax(score, axis=1)
        # print("max_index: ",  max_index, "\nlen(imgs): ", len(imgs))

        for i,id in enumerate(max_index):
            print(f'in {face[i]}')
            if(score[i][id] >= 0.3):
                img_with_text = aligned_bgr_imgs[i]
                face_text = f'{ID_list[int(id/3)]}-Score:{score[i][id]:.2f}'
                cv2.putText(img_with_text, face_text, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
                display(Image(data=cv2.imencode('.png', img_with_text)[1].tobytes(), format='png'))
                if cur_time - cam_start_time >= 5:
                    cam_start_time = cur_time
                    student_arrive(str(ID_list[int(id/3)]))

            else:
                print("no ensured answer")
                img_with_text = aligned_bgr_imgs[i]
                face_text = "no ensured answer"
                cv2.putText(img_with_text, face_text, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
                display(Image(data=cv2.imencode('.png', img_with_text)[1].tobytes(), format='png'))
            yoloface.plot_one_box(coordinate, frame)
    else:
        with torch.no_grad():
            similarity_scores = torch.cat(features) @ torch.cat(features).T
        print(similarity_scores)

        # imgs=[]
    cv2.imshow('Video Stream', frame)
    # detect(opt=opt)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


ValueError: not enough values to unpack (expected 2, got 1)

# ORIGINAL CODE

In [ ]:
# source = 'test_images'
# features = []
# face=[]
# imgs=[]
# test_image = sorted(os.listdir(source))
# ct=0
# for fname in test_image:
#     path = os.path.join(source, fname)
#     if isyolo:
#         aligned_bgr_imgs,tt = yoloface.show_detect(path)
#         for i,img in enumerate(aligned_bgr_imgs) :
#             imgs.append(tt[i])
#             # cv2.imwrite(f'deteced_{ct}_{i}.jpg',img)
#             bgr_tensor_input = to_input(img,True).to(device)
#             feature, _ = model(bgr_tensor_input)
#             features.append(feature)
#             face.append(f'deteced_{ct}_{i}.jpg')
#         ct+=1
#     else:
#         aligned_rgb_img = align.get_aligned_face(path, isarray=True)
#         if(aligned_rgb_img is not None):
#             bgr_tensor_input = to_input(aligned_rgb_img,False).to(device)
#             feature, _ = model(bgr_tensor_input)
#             features.append(feature)

In [ ]:
# from IPython.display import display, Image

# database_dir =  '/content/drive/MyDrive/AdaFace/face_database'
# database_pt_dir = os.path.join(database_dir,'database.pt')
# if(os.path.exists(database_pt_dir)):
#     database = torch.load(database_pt_dir).to(device)
#     with open(os.path.join(database_dir, 'ID_list.txt'), 'r') as file:
#         ID_list = [line.strip() for line in file.readlines()]
#     start_time = time.time()
#     with torch.no_grad():
#         similarity_scores = torch.cat(features) @ database.T
#     end_time = time.time()
#     execution_time = end_time - start_time
#     print(f"computing time: {execution_time} seconds")
#     score = similarity_scores.cpu().detach().numpy()
#     max_index = np.argmax(score, axis=1)

#     for i,id in enumerate(max_index):
#         print(f'in {face[i]}')
#         if(score[i][id] >= 0.3):
#             img_with_text = imgs[i].copy()
#             face_text = f'{ID_list[int(id/3)]}-Score:{score[i][id]:.2f}'
#             cv2.putText(img_with_text, face_text, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
#             # plt.figure(figsize=(12,4))
#             # img_with_text_rgb = cv2.cvtColor(img_with_text, cv2.COLOR_BGR2RGB)
#             display(Image(data=cv2.imencode('.png', img_with_text)[1].tobytes(), format='png'))
#             student_arrive(str(ID_list[int(id/3)]))
#         else:
#             print("no ensured answer")
# else:
#     with torch.no_grad():
#         similarity_scores = torch.cat(features) @ torch.cat(features).T
#     print(similarity_scores)